<a href="https://colab.research.google.com/github/mmsamiei/acv_final_project/blob/main/coil_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://www.cs.columbia.edu/CAVE/databases/SLAM_coil-20_coil-100/coil-100/coil-100.zip

--2021-02-09 21:45:40--  http://www.cs.columbia.edu/CAVE/databases/SLAM_coil-20_coil-100/coil-100/coil-100.zip
Resolving www.cs.columbia.edu (www.cs.columbia.edu)... 128.59.11.206
Connecting to www.cs.columbia.edu (www.cs.columbia.edu)|128.59.11.206|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.cs.columbia.edu/CAVE/databases/SLAM_coil-20_coil-100/coil-100/coil-100.zip [following]
--2021-02-09 21:45:41--  https://www.cs.columbia.edu/CAVE/databases/SLAM_coil-20_coil-100/coil-100/coil-100.zip
Connecting to www.cs.columbia.edu (www.cs.columbia.edu)|128.59.11.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130688843 (125M) [application/zip]
Saving to: ‘coil-100.zip’

coil-100.zip        100%[===================>] 124.63M  15.6MB/s    in 9.3s    

2021-02-09 21:45:51 (13.4 MB/s) - ‘coil-100.zip’ saved [130688843/130688843]



In [ ]:
!unzip /content/coil-100.zip -d . 

In [102]:
!pip install timm

     |████████████████████████████████| 245kB 5.4MB/s 


In [7]:
import glob
from PIL import Image

In [79]:
train_data_list = []
test_data_list = [] 

for file_name in glob.iglob('/content/coil-100/*.png'):
  im = Image.open(file_name)
  id = file_name.split('/')[-1].split('.')[0]
  degree = int(id.split('__')[1])
  label = int(id.split('__')[0][3:])-1
  if degree <= 90:
    train_data_list.append([im, degree, label])
  else:
    test_data_list.append([im, degree, label])

In [80]:
len(test_data_list)

5300

In [118]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import timm
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm

In [91]:
class CoilDataset(Dataset):

  def __init__(self, data_list, transform=None):
    self.data_list = data_list
    self.transform = transform

  def __len__(self):
    return len(self.data_list)
  
  def __getitem__(self, idx):


    sample = {'image': self.data_list[idx][0], 'label': self.data_list[idx][1]}

    sample['image'] = self.transform(sample['image'])

    return sample

In [92]:
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = CoilDataset(train_data_list, transform=transform)
test_dataset = CoilDataset(test_data_list, transform=transform)

In [98]:
train_dataloader = DataLoader(train_dataset, batch_size=16,
                        shuffle=True, num_workers=0)

test_dataloader = DataLoader(test_dataset, batch_size=16,
                        shuffle=True, num_workers=0)

In [109]:
class CNNNet(torch.nn.Module):
  def __init__(self):
    super(CNNNet, self).__init__()
    self.cnn = timm.create_model('resnet50', pretrained=True, num_classes=0)
    self.fc = torch.nn.Linear(2048, 100)
    self.cnn = self.cnn.eval()
    for param in self.cnn.parameters():
      param.requires_grad = False
    nn.init.xavier_normal_(self.fc.weight)

  def forward(self, x):
    # x = [batch, channel, 224, 224]
    temp = self.cnn(x)
    temp = self.fc(temp)
    return temp

class TransformerNet(torch.nn.Module):
  def __init__(self):
    super(TransformerNet, self).__init__()
    self.transformer = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=0)
    self.fc = torch.nn.Linear(768, 100)
    self.transformer = self.transformer.eval()
    for param in self.transformer.parameters():
      param.requires_grad = False
    nn.init.xavier_normal_(self.fc.weight)

  def forward(self, x):
    # x = [batch, channel, 224, 224]
    temp = self.transformer(x)
    temp = self.fc(temp)
    return temp


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        xavier(m.bias.data)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [110]:
dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [111]:
x = torch.rand(4, 3, 224, 224)
mcnn = CNNNet().to(dev)
mtra = TransformerNet().to(dev)

count_parameters(mcnn), count_parameters(mtra)

(204900, 76900)

In [115]:
criterion = nn.CrossEntropyLoss()

In [126]:
def evaluate(model): 
  correct = 0
  total = 0
  model = model.eval()

  with torch.no_grad():
    for i, batch in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
      batch_x = batch['image'].to(dev).float()
      labels = batch['label'].to(dev)
      y_pred = model(batch_x)
      _, predicted = torch.max(y_pred.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  
  return correct / total

In [127]:
optimizer = optim.Adam(params =mcnn.fc.parameters() , lr=1e-4)

MAX_EPOCH = 5

for epoch in range(MAX_EPOCH):
  running_loss = 0.0

  for i, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
    optimizer.zero_grad()
    batch_x = batch['image'].to(dev)
    batch_y = batch['label'].to(dev)
    y_pred = mcnn(batch_x.float())
    loss = criterion(y_pred, batch_y)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
    if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
    break
  accuracy = evaluate(mcnn)
  mcnn = mcnn.train()
  print('epoch {}, accuracy: {}'.format(epoch, accuracy))


KeyboardInterrupt: ignored